## Activity 5 - Travelling Salaesman Problem using GA
by Francis John N. Magallanes and John Matthew Vong

This jupyter notebook will follow the implementation in this [website](https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3), geeks for geeks [website](geeksforgeeks.org/traveling-salesman-problem-using-genetic-algorithm), this [journal](http://www.jcreview.com/fulltext/197-1578037726.pdf?1578371869&fbclid=IwAR3dtCTyZE5FlArzuNRANS3guNg9OBPLPxDQVMogfkizO6BkPrUkhEC9X3Y), and this [youtube video](https://www.youtube.com/watch?v=nhT56blfRpE). Object Oriented Programming will be incorporated in the implementation especially on the representation of the chromosomes

There will 5 functions to be implemented that will do the following:

1. Initialization of the Population
2. Fitness function
3. Selection
4. Crossover
5. Mutation


As for the algorithmic implementation, it will follow the geeks for geeks [website](geeksforgeeks.org/traveling-salesman-problem-using-genetic-algorithm) and the tutorialspoint [website](https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_fundamentals.htm). Below shows the algorithm to be used in this implementation:

1. Initialize the population randomly.
2. Determine the fitness value of the initial chromosome.
3. Until termination criteria is achieved:
    1. Select parents and perform crossover.
    2. Perform mutation.
    3. Calculation of the fitness function
    4. Survival Selection

In [79]:
#libraries used for this genetic algorithm
import random

### Travelling Salesman Problem

The problem states that the salesman should visit every node once and return to the starting node

Graph of our Problem:

![graph of the travelling salesman problem](graph.png)

In [80]:
#Representation of the graph above with the corresponding cost
#it will use "adjacency list" for the representation
#note this will be the basis for the fitness function

adj_list = {
    #for the node A and its connections with the connection cost
    "A" : {"B" : 3 , "C" : 6, "D" : 4, "E" : 9, "G" : 12},

    #for the node B and its connections with the connection cost
    "B" : {"A" : 3, "C": 5, "G": 6, "F" : 6},

    #for the node C and its connections with the connection cost
    "C" : {"A" : 6, "B" : 5, "D" : 2, "E" : 7, "F" : 4},

    #for the node D and its connections with the connection cost
    "D" : {"A" : 4, "C" : 2, "E" : 8, "F" : 7},

    #for the node E and its connections with the connection cost
    "E" : {"A" : 9, "C" : 7, "D" : 8, "F" : 3, "G" : 5},

    #for the node F and its connections with the connection cost
    "F" : {"B" : 6, "C" : 4, "D" : 7, "E" : 3},

    #for the node G and its connections with the connection cost
    "G" : {"A" : 12, "B" : 6, "E" : 5}
}

### Chromosome Representation

The number of genes will be n -1 (where n is number of nodes in the graph) or 6 genes. Each chromosome represent a possible path or solution to the travelling sales man problem. Using OOP, the calculation of the fitness function and the mutation of the chromosome will be  incorporated to the class

In [81]:
class Chromosome_TSP():

    #the adj_list should contain the graph representation of TSP with weights
    def __init__(self,adj_list : dict, chromosome:list = None, src: str = 'A'):
        
        self.adj_list = adj_list

        if chromosome is None:
            #this will initialize chromosome through shuffling
            self.chromosome = list(self.adj_list)
            self.chromosome.remove(src)
            random.shuffle(self.chromosome)
        else:
            self.chromosome = chromosome
    
    #for the mutation of the chromosome
    #it will use swap mutation
    def mutation(self, mutation_rate : float = 0.1):
        if random.random() <= mutation_rate :
            i1, i2 = random.sample(range(6), 2) #randomly choose the places to be swapped
            self.chromosome[i1], self.chromosome[i2] = self.chromosome[i2],self.chromosome[i1]
    
    #for the calculation of the fitness of the instance of the chromosome 
    def calculate_fitness(self) -> int:
        cost_path = 0  #this is for the results of the fitness
        temp = ["A"] + self.chromosome + ["A"] # this will be essential for the computation

        for i in range(len(self.chromosome)):
            #this will check whether temp [i] and temp[i+1] is a valid edge
            #if not, it will break the loop and the cost will be 100000 (or any larger values)
            if temp[i + 1] in self.adj_list[temp[i]].keys():
                cost_path = cost_path + self.adj_list[temp[i]][temp[i+1]]
            else:
                cost_path = 100000
                break
        
        return cost_path


### Initialization of the Population Function


In [82]:
def init_population(population_size) -> list:
    return [Chromosome_TSP(adj_list) for _ in range(population_size)]


### Crossover Function

It will implement the Single Point Crossover. Click [here](https://www.ripublication.com/ijcir17/ijcirv13n7_15.pdf) for the reference implementation

In [83]:
# this will implement the single point crossover
def crossover_func(parent1 : Chromosome_TSP, parent2 : Chromosome_TSP ) -> Chromosome_TSP :

    #randomly choose the crossover points
    #with this, there is will be a min of 1 item and max of 5 items for crossover
    crossover_pnt = random.randint(2,5)
    
    print(crossover_pnt)

#copy the chromosome list from the parent to preserve the list of the parent's chromosome
    c1 = parent1.chromosome.copy()
    c2 = parent2.chromosome.copy()
    
    #the crossover proper
    #child 1 chromosome
    #this will initialize the chromosome and put the parent 2 chromosome
    child1_chromosome = ["" for _ in range(crossover_pnt)] + c2 [crossover_pnt:]

    #this will put the genes from parent1 to the child1
    for i in range(crossover_pnt):
        #this will avoid the duplicates
        if not (c1[i] in  c2 [crossover_pnt:]) :
            child1_chromosome[i] = c1[i]

    #this is for the unfilled genes due to duplicates
    if "" in child1_chromosome:
        for i in range(len(child1_chromosome)):
            
            if child1_chromosome[i] == "":

                #finding the another appropreaite gene
                for gene in c1[crossover_pnt:] :
                    if not (gene  in child1_chromosome):
                        child1_chromosome[i] = gene
                        break

    #child 2 chromosome
    #this will initialize the chromosome and put the parent 2 chromosome
    child2_chromosome = ["" for _ in range(crossover_pnt)] + c1[crossover_pnt:]

    #this will put the genes from parent1 to the child1
    for i in range(crossover_pnt):
        #this will avoid the duplicates
        if not (c2[i] in  c1[crossover_pnt:]) :
            child2_chromosome[i] = c2[i]

    #this is for the unfilled genes due to duplicates
    if "" in child2_chromosome:
        for i in range(len(child2_chromosome)):
            
            if child2_chromosome[i] == "":

                #finding the another appropreaite gene
                for gene in c2[crossover_pnt:] :
                    if not (gene  in child2_chromosome):
                        child2_chromosome[i] = gene
                        break

    return Chromosome_TSP(parent1.adj_list, child1_chromosome), Chromosome_TSP(parent2.adj_list, child2_chromosome)

In [84]:
parent1 = Chromosome_TSP(adj_list)
parent2 = Chromosome_TSP(adj_list)

print(parent1.chromosome)
print(parent2.chromosome)

child1, child2 = crossover_func(parent1, parent2)

print(child1.chromosome)
print(child2.chromosome)

['C', 'F', 'B', 'G', 'D', 'E']
['E', 'C', 'F', 'G', 'D', 'B']
2
['C', 'E', 'F', 'G', 'D', 'B']
['F', 'C', 'B', 'G', 'D', 'E']


### Selection Function

In [86]:
#elitism will be implemented for the selection functin
#Top Lowest will be basis of the elitism
#n refers on the number of chromosomes to choosen for the next generation
def selection_func (population : list, n : int) -> list :
    
    if len(population) < n:
        raise Exception("number of chromosomes to be choosen is greater than the size of the population")

    #evaluate the population based on its fitness function
    popu_eval = (c.calculate_fitness() for c in population)

    # this will combine the chromosome and its evaluation of the whole population
    popu_with_eval = list(zip(population,popu_eval)) 

    #this will sort the population according on its evaluation
    popu_with_eval.sort(key = lambda i : i[1])

    return popu_with_eval[:n] #this will only the bottom n chromosomes 

In [88]:
p =  init_population(100)

selected = selection_func(p,5)

for s in selected:
    print (s[1])

25
25
25
28
32


In [35]:
d =  {}
d[34] = 2
d[43] = 5
d[4]= 4

sorted(d)
d

{34: 2, 43: 5, 4: 4}